In [1]:
import pandas as pd 

In [118]:
crime_ = pd.read_csv('crime_scene_report.csv')
driver_ = pd.read_csv('drivers_license.csv')
facebook_ = pd.read_csv('facebook_event_checkin.csv')
get_fit_in = pd.read_csv('get_fit_now_check_in.csv')
get_fit_mem = pd.read_csv('get_fit_now_member.csv')
income = pd.read_csv('income.csv')
interview = pd.read_csv('interview.csv')
person = pd.read_csv('person.csv')

In [32]:
crime_.head(10)


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipis...",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows ...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system...,SQL City


In [44]:
len(crime_[crime_["city"] == "SQL City"])

9

Посмотрим на преступления в нашем городе

In [36]:
crime_[crime_["city"] == "SQL City"]

,date,type,description,city
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
7,20170712,theft,"A lone hunter stalks the night, firing arrows ...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system...,SQL City
10,20180103,bribery,"Apparently, Cayde thought it necessary to expo...",SQL City
11,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
12,20180115,assault,Report Not Found,SQL City
1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


Нас интересует убийство

In [344]:
crime_[(crime_["city"] == "SQL City") & (crime_["type"] == "murder")]

,date,type,description,city
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


Зная дату убийства хотим понять сколько их было, найти зацепку-бы

In [56]:
crime_[(crime_["city"] == "SQL City") & (crime_["type"] == "murder") & (crime_["date"] == 20180115)]

,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [181]:
crime_[(crime_["city"] == "SQL City") & (crime_["type"] == "murder") & (crime_["date"] == 20180115)].reset_index(drop=True)["description"][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

Свидетель # 2 (Annabel)

In [200]:
person[(person["name"].str.contains("Annabel")) & (person["address_street_name"] == "Franklin Ave")]


,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [244]:
id_2 = person[(person["name"].str.contains("Annabel")) & (person["address_street_name"] == "Franklin Ave")].reset_index(drop=True)["id"][0]

Свидетель # 1 ("Northwestern Dr"), который живет в последнем доме

In [275]:
person[(person["address_street_name"] == "Northwestern Dr")].sort_values(by="address_number", ascending=False).reset_index(drop=True).iloc[[0]]

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [248]:
id_1 = person[(person["address_street_name"] == "Northwestern Dr")].sort_values(by="address_number", ascending=False).reset_index(drop=True).iloc[[0]]["id"][0]

Найдем их показания

In [283]:
interview.head(10)

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for ..."
4,33856,\n
5,82799,\n
6,66493,would be offended again.\n
7,46263,"‘So he did, so he did,’ said the Gryphon, sigh..."
8,54263,very glad that it was over at last: ‘and I do ...
9,62764,come to the end: then stop.’\n


In [285]:
interview[interview["person_id"].isin([id_2, id_1])]

,person_id,transcript
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."


In [287]:
interview[interview["person_id"].isin([id_2, id_1])].reset_index(drop=True)["transcript"][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [291]:
interview[interview["person_id"].isin([id_2, id_1])].reset_index(drop=True)["transcript"][1]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

Теперь зная дату check_in и membership_id 48Z пойдем в зал

In [298]:
get_fit_in[(get_fit_in["membership_id"].str.contains("48Z")) & (get_fit_in["check_in_date"] == 20180109)]

,membership_id,check_in_date,check_in_time,check_out_time
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


In [302]:
gym = get_fit_in[(get_fit_in["membership_id"].str.contains("48Z")) & (get_fit_in["check_in_date"] == 20180109)]

In [300]:
get_fit_mem.head(10)

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver
5,6W6I2,69510,Paris Accurso,20170613,regular
6,5Y28Y,15218,Millicent Yessios,20180205,gold
7,156A7,79701,Marcelino Leard,20180428,silver
8,A5N3S,24541,Mary Cameron,20170501,gold
9,X0643,15247,Shondra Ledlow,20170521,silver


Вообще было бы неплохо понять, кто это такие по номерам членства 

In [308]:
get_fit_mem.merge(gym, left_on="id", right_on="membership_id")

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


Итак, 2 подозреваемых 2018-01-09 были в зале, с id 48Z и стаутос GOLD MEMBER, знаем, что один из них обязательно должен быть мужчиной и его номер машины включает H42W 

In [311]:
driver_.head(10)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS
5,101494,48,55,blue,red,female,81X1N7,Kia,Sportage
6,101568,53,78,brown,blue,male,SJ57LL,Saab,9-5
7,101586,57,70,amber,brown,male,31U1KE,Jaguar,XK
8,101611,40,65,blue,white,female,5O8VW7,GMC,Sierra Denali
9,101613,81,58,brown,grey,female,5PCQB5,GMC,Sierra 2500


Проверяем по базе водителей

In [321]:
driver_[(driver_["plate_number"].str.contains("H42W")) & (driver_["gender"] == "male")]

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima


Провериам по номеру лицензии

In [324]:
person.head(10)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
5,10027,Antione Godbolt,439509,2431,Zelham Dr,491650087
6,10034,Kyra Buen,920494,1873,Sleigh Dr,332497972
7,10039,Francesco Agundez,278151,736,Buswell Dr,861079251
8,10095,Leslie Thate,729987,2772,Camellia Park Circle,127944356
9,10122,Alva Conkel,779002,116,Diversey Circle,148521773


In [326]:
driver = driver_[(driver_["plate_number"].str.contains("H42W")) & (driver_["gender"] == "male")]

In [328]:
person.merge(driver, left_on="license_id", right_on="id")

,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,51739,Tushar Chandra,664760,312,Phi St,137882671,664760,21,71,black,black,male,4H42WR,Nissan,Altima
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


И наконец соберем воедино водителя и его похождения в зал

In [335]:
witness = get_fit_mem.merge(gym, left_on="id", right_on="membership_id")
person_ = person.merge(driver, left_on="license_id", right_on="id")

Итак, наш подозреваемый убийца.....

In [337]:
murder = person_.merge(witness, left_on="name", right_on="name")

In [339]:
murder["name"]

0    Jeremy Bowers
Name: name, dtype: object

Мы нашли полозреваемого убийцу, ура! 
Еще бы проверить на правильность!

Посмотрим на его показания

In [369]:
interview[interview["person_id"] == murder["id_x"][0]]["transcript"].reset_index(drop=True)[0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

Итак, его наняли => подозреваемый и правда является убийцей, дополнительно можно было бы найти кто его нанял.